In [1]:
import os, copy, torch, numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.transforms import v2
from torchvision import transforms
import torch
import torch.nn as nn
from tqdm import tqdm 
from PIL import Image
import pandas as pd

from matplotlib import pyplot as plt

device = "cpu"

model = torch.jit.load('contour_model.pt')
model.eval();

In [13]:
src = "dataset/rs/"

imgs = os.listdir(src)
if "back" in imgs:
    imgs.remove("back")

props = []

model.train()
for i in imgs:

    toTensor = transforms.ToTensor()
    img = Image.open(src + i)
    img = toTensor(img.convert("RGBA"))
    
    alpha = img[[3]]
    img = img[[0,1,2]] * alpha
    img += 1 - alpha

    out = model(img[None, ...])[0]

    rzero = (out < 0.3)
    rones = (out > 0.5)
    rnone = (out >= 0.3) * (out <= 0.7)

    prop = rnone.sum() / torch.clamp(rones + rnone, 0, 1).sum()
    if prop > 0.16:
        props.append((float(prop), i))

    #plt.imshow(torch.clamp(ro, 0, 1)[0].detach().numpy())
    #plt.show()
    #plt.imshow(torch.clamp(dv, 0, 1)[0].detach().numpy())
    #plt.show()

pd.DataFrame(props).to_csv("pds.csv")

In [14]:
len(props)

36

In [17]:
src = "dataset/rs_cont/"
dst = "dataset/gt_rs/"
os.makedirs(dst, exist_ok=True)

for _, p in props:
    src + p

    os.system(f'cp "{src + p}" "{dst + p}"')  
